In [1]:
partition = 100

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:

n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-verbose', '0',
        '-jointly_training'
    ]
    
    _, _, _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")



Running: n_tree=5, tree_depth=3, batch_size=256
Use gtd100 dataset


Training Epochs:  26%|██▌       | 51/200 [00:17<00:51,  2.91it/s]

Early stopping at epoch 52

Running: n_tree=5, tree_depth=3, batch_size=512
Use gtd100 dataset



Training Epochs:  43%|████▎     | 86/200 [00:20<00:26,  4.27it/s]

Early stopping at epoch 87

Running: n_tree=5, tree_depth=3, batch_size=1000
Use gtd100 dataset


...

Early stopping at epoch 14



Running: n_tree=20, tree_depth=10, batch_size=1000
Use gtd100 dataset


Training Epochs:   7%|▋         | 14/200 [00:08<01:51,  1.67it/s]

Early stopping at epoch 15



Best hyperparameter configuration:
{'n_tree': 15, 'tree_depth': 10, 'batch_size': 256}
Best accuracy: 0.528889


In [4]:
sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', str(best_config['n_tree']),
    '-tree_depth', str(best_config['tree_depth']),
    '-batch_size', str(best_config['batch_size']),
    '-epochs', '1000',
    '-verbose', '1',
    '-jointly_training'
]

best_model, preds, targets, labels, epoch_logs = main()


Use gtd100 dataset


Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Train Epoch: 1 [0/2100 (0%)]	Loss: 3.401032


Training Epochs:   0%|          | 1/1000 [00:01<20:39,  1.24s/it]

Train Epoch: 2 [0/2100 (0%)]	Loss: 3.399657


...

Train Epoch: 154 [0/2100 (0%)]	Loss: 1.379110


Training Epochs:  15%|█▌        | 154/1000 [02:54<16:21,  1.16s/it]

Train Epoch: 155 [0/2100 (0%)]	Loss: 1.352283


Training Epochs:  15%|█▌        | 154/1000 [02:55<16:04,  1.14s/it]

Early stopping at epoch 155


In [5]:
from sklearn.metrics import classification_report

print(classification_report(targets, preds))

              precision    recall  f1-score   support

           0       0.54      0.70      0.61        30
           1       0.68      0.87      0.76        30
           2       0.53      0.80      0.64        30
           3       0.40      0.57      0.47        30
           4       0.45      0.57      0.50        30
           5       0.67      0.80      0.73        30
           6       0.46      0.43      0.45        30
           7       0.33      0.40      0.36        30
           8       0.88      0.93      0.90        30
           9       0.73      0.73      0.73        30
          10       0.53      0.93      0.67        30
          11       0.51      0.60      0.55        30
          12       0.62      0.53      0.57        30
          13       0.91      0.67      0.77        30
          14       0.62      0.27      0.37        30
          15       0.13      0.13      0.13        30
          16       0.38      0.60      0.46        30
          17       1.00    

In [6]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [7]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd100 to results/confusion_matrix_partition_gtd100.png
